In [1]:
import sys
sys.path.append("/home/gerald/Documents/CPD/repository/LifelongInformationRetrieval") 

In [2]:
from lire.data_tools.dataset import MSMarco
from lire.data_tools.data_reader import Qrels
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
import pickle
import tqdm

In [3]:
data_folder    = "/local/gerald/CPD/data"
embedding_path = "/local/gerald/CPD/data/query_embeddings2.pkl"
cluster_path   = "/local/gerald/CPD/data/cluster_completed2.pth"
cluster_center   = "/local/gerald/CPD/data/cluster_center2.pth"

train_queries = MSMarco.MSMarcoPassageRankingDataset.load_queries("train", data_folder)
dev_queries   = MSMarco.MSMarcoPassageRankingDataset.load_queries("dev", data_folder)
eval_queries  = MSMarco.MSMarcoPassageRankingDataset.load_queries("eval", data_folder)
train_qrels   = MSMarco.MSMarcoPassageRankingDataset.load_qrels("train", data_folder)
dev_qrels     = MSMarco.MSMarcoPassageRankingDataset.load_qrels("dev", data_folder)
qrels_set     = Qrels.merge_qrels(train_qrels, dev_qrels)

queries_set   = pd.concat([train_queries, dev_queries, eval_queries])
query_example = queries_set.iloc[11][1]
print('Example of query: "', query_example, '0"')

Example of query: " highmark address 0"


In [4]:
queries_sentences  = queries_set[1].tolist() 
queries_ids        = queries_set.index.values.tolist()
queries_inv_ids    = {v:i for i, v in enumerate(queries_ids)}
queries_ids_train  = len(train_queries)
queries_ids_dev    = len(train_queries) + len(dev_queries)
queries_ids_eval   = len(train_queries) + len(dev_queries) + len(eval_queries)


clusters    = torch.load(cluster_path)


In [5]:
clusters    = torch.load(cluster_path)

In [6]:
len(clusters)

3551

In [7]:


    train_set, dev_set, eval_set = [], [], [] 
    for cluster in clusters:
        train_set.append([])
        dev_set.append([])
        eval_set.append([])

        for query in cluster:
            if(str(queries_ids[query]) in qrels_set):
                if(query < queries_ids_train):
                    train_set[-1].append(queries_ids[query])
                elif(query < queries_ids_dev):
                    dev_set[-1].append(queries_ids[query])
                elif(query < queries_ids_eval):
                    eval_set[-1].append(queries_ids[query])
                else:
                    raise Exception("Irrelevant query id")

    

In [8]:
print(min([len(i) for i in train_set]))
print(min([len(i) for i in dev_set]))
print(min([len(i) for i in eval_set]))

8
0
0


In [9]:
too_small_dev_cluster = [ i for i, cluster in enumerate(dev_set) if(len(cluster) <= 2)]
too_small_eval_cluster = [ i for i, cluster in enumerate(eval_set) if(len(cluster) <= 2)]

print("Number of dev sets that contains less than 2 queries : ", len(too_small_dev_cluster))
print("Number of eval sets that contains less than 2 queries : ", len(too_small_eval_cluster))

Number of dev sets that contains less than 2 queries :  149
Number of eval sets that contains less than 2 queries :  3551


In [10]:

with open(embedding_path, "rb") as fIn:
    cache_data = pickle.load(fIn)
corpus_sentences = cache_data['sentences']
corpus_embeddings = cache_data['embeddings']

clusters_to_merge = too_small_dev_cluster

In [11]:
print([len(clusters[cluster]) for cluster in clusters_to_merge])

[124, 81, 166, 34, 90, 136, 162, 114, 120, 85, 139, 52, 119, 155, 87, 130, 161, 118, 72, 39, 86, 47, 61, 117, 47, 62, 69, 94, 77, 89, 96, 52, 154, 57, 144, 54, 97, 137, 66, 52, 76, 72, 75, 124, 44, 21, 37, 47, 88, 94, 82, 140, 90, 83, 66, 61, 60, 105, 70, 44, 94, 90, 93, 110, 55, 66, 90, 104, 86, 40, 35, 90, 85, 114, 99, 56, 119, 65, 104, 72, 164, 115, 84, 57, 59, 42, 86, 41, 67, 58, 60, 91, 29, 65, 85, 84, 144, 79, 24, 110, 53, 36, 113, 61, 153, 46, 105, 47, 50, 125, 128, 50, 113, 126, 131, 126, 94, 67, 85, 67, 94, 73, 87, 60, 53, 61, 98, 35, 68, 89, 100, 110, 56, 63, 52, 36, 52, 43, 70, 90, 146, 36, 120, 23, 75, 121, 66, 78, 15]


In [12]:
print("Number of cluster to merge : ", len(clusters_to_merge))

Number of cluster to merge :  149


In [13]:
cluster_center = torch.load("/local/gerald/CPD/data/cluster_center2.pth")
print(len(cluster_center))
print(len(clusters))


3551
3551


In [14]:
cluster_center_merged[0]

NameError: name 'cluster_center_merged' is not defined

In [15]:
clusters_center = torch.load("/local/gerald/CPD/data/cluster_center2.pth")
cluster_center_merged  = []
cluster_center_reindex = []
new_clusters = []

# reindexing
for i, cluster_center in enumerate(clusters_center):
    if i not in clusters_to_merge:
        cluster_center_merged.append(clusters_center[i])
        cluster_center_reindex.append(i)
        new_clusters.append(clusters[i])

# much faster using pytorch
cluster_center_merged  = torch.Tensor(cluster_center_merged)
corpus_embeddings = torch.Tensor(corpus_embeddings)
print(len(torch.Tensor(cluster_center_merged)))
print(len(cluster_center_reindex))
print(len(new_clusters))
# merging
for i, cluster_index in zip(tqdm.notebook.trange(len(clusters_to_merge)), clusters_to_merge):
    cluster = clusters[cluster_index]
    for query in cluster:

        cos_sim = util.pytorch_cos_sim(cluster_center_merged, corpus_embeddings[query])
        new_clusters[cos_sim.argmax()].append(query)
        #sp = new_clusters[cos_sim.argmax()][0]
        #print('query to move"' , corpus_sentences[query], '" | sample of new cluster "', corpus_sentences[sp] , '"\n')        


        

3402
3402
3402


  0%|          | 0/149 [00:00<?, ?it/s]

In [16]:
train_set, dev_set, eval_set = [], [], [] 
for cluster in new_clusters:
    train_set.append([])
    dev_set.append([])
    eval_set.append([])
    
    for query in cluster:
        if(str(queries_ids[query]) in qrels_set):
            if(query < queries_ids_train):
                train_set[-1].append(queries_ids[query])
            elif(query < queries_ids_dev):
                dev_set[-1].append(queries_ids[query])
            elif(query < queries_ids_eval):
                eval_set[-1].append(queries_ids[query])
            else:
                raise Exception("Irrelevant query id")

In [17]:
print(min([len(i) for i in train_set]))
print(min([len(i) for i in dev_set]))
print(min([len(i) for i in eval_set]))

11
3
0


In [18]:
import json
with open('/web/gerald/public_html/lire_data/msmarco_topics_cleaned_train_stsb-roberta-large.json', 'w') as outfile:
    json.dump(train_set, outfile, indent = 4)
with open('/web/gerald/public_html/lire_data/msmarco_topics_cleaned_dev_stsb-roberta-large.json', 'w') as outfile:
    json.dump(dev_set, outfile, indent = 4)
with open('/web/gerald/public_html/lire_data/msmarco_topics_cleaned_eval_stsb-roberta-large.json', 'w') as outfile:
    json.dump(eval_set, outfile, indent = 4)